### Introduction
This project aims to develop classification models to analyze subscribers' behavior and predict user plans. The models will be used to recommend subscribers of Megaline's legacy plans, one of their newer plans, Smart, or Ultra. 

Using the `users_behavior.csv` dataset, I tested various models with different hyperparameters. The models' threshold of success is 0.75 or 75%.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.duplicated().sum()

0

In [6]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [7]:
# Splitting the dataset into features and target, then into train, validation, and test set(3:1:1 split).
features = df.drop('is_ultra', axis= 1)
target= df['is_ultra']
features_train,temp_train, target_train,temp_target= train_test_split(features, target, test_size= 0.40, random_state=42) 
features_validation, features_test, target_validation, target_test= train_test_split(temp_train, temp_target, test_size=0.50, random_state=42) 

### Explanation 

The lesson mentioned a `3:1:1` (60, 20, 20) split being the best, so this was the logic used in my split. I started with a 60/40 split for a training and temporary set, then followed up by splitting the temporary set in half.

In [9]:
# Finding the best hyperparameters for a Decision Tree model 
best_decision_model= None
best_depth= 0
best_results= 0

for depth in range(1,6):
    model=DecisionTreeClassifier(random_state=42, max_depth=depth)
    model.fit(features_train, target_train)
    predictions= model.predict(features_validation)
    results = accuracy_score(target_validation, predictions)
    if results > best_results:
        best_decision_model= model
        best_depth= depth
        best_results= results
print(f'Best Depth: {best_depth} Accuracy Score: {best_results}')  

Best Depth: 3 Accuracy Score: 0.7916018662519441


In [10]:
# Assessing the quality of the best decision tree model on the test set
test_predictions= best_decision_model.predict(features_test)
test_accuracy_score= accuracy_score(target_test,test_predictions)
print("The test set's accuracy score is:", test_accuracy_score)

The test set's accuracy score is: 0.8055987558320373


### Evaluation

The Decision Tree model with a depth of 3 achieved a validation accuracy of `79.2%` and a test accuracy of `80.6%`, both exceeding the assignment's required threshold of 75%. The higher test accuracy compared to the validation set suggests that the model generalizes well to unseen data without signs of overfitting or underfitting. Overall, the decision tree demonstrates strong performance on this task.

In [11]:
# Finding the best estimators for a Random Forest model
best_rf_model= None
best_est= 0
best_score= 0

for est in range(1, 8):
    model= RandomForestClassifier(n_estimators=est, random_state=42)
    model.fit(features_train, target_train)
    score= model.score(features_validation, target_validation)
    if score > best_score:
        best_rf_model = model
        best_est = est
        best_score= score
print(f'Best number of estimators: {best_est} Best Score: {best_score}')

Best number of estimators: 4 Best Score: 0.7916018662519441


In [12]:
# Assessing the quality of the best random forest model on the test set
test_score = best_rf_model.score(features_test, target_test)
print(f"The test set's score is:", test_score)

The test set's score is: 0.8009331259720062


### Evaluation

The Random Forest model with an estimators count of 4 achieved a validation accuracy of `79.2%` and a test accuracy of `80.1%`, performing nearly identical to the decision tree model while maintaining a consistent performance on unseen data. The validation and test scores suggest the model generalizes well without signs of overfitting or underfitting. Since the model surpasses the assignment’s target threshold of 75%, it can be considered successful and reliable for this task.

In [13]:
# Using a Logistic Regression model 
lr_model = LogisticRegression(solver='liblinear', random_state=42)
lr_model.fit(features_train, target_train)
lr_score = lr_model.score(features_validation, target_validation)
print('Logistic Regression Score:', lr_score)

Logistic Regression Score: 0.7076205287713841


In [14]:
# Using the LR model on the test set
test_score= lr_model.score(features_test, target_test)
print(f'LR model score on the test set:', test_score)

LR model score on the test set: 0.7076205287713841


### Evaluation

The logistic regression model produced identical accuracy scores of `70.76%` on both the validation and test sets, indicating strong consistency and generalization with no signs of overfitting or underfitting. However, this model missed the assignment’s required accuracy threshold of 75%, making it the weakest performer among the models tested. This result suggests that logistic regression may not be complex enough to capture the underlying patterns in this particular dataset. 

### Final Thoughts

Based on the parameters of the assignment, I was tasked with developing several classification models, using accuracy as the sole evaluation metric. Given this constraint, both the decision tree and random forest models performed well, with very similar results. However, if I had to choose one, I would choose to deploy the decision tree, as it achieved slightly better accuracy on unseen test data.

Going forward, I would consider testing additional hyperparameters and incorporating other evaluation metrics, such as precision, recall, or F1-score, to gain more understanding of the models' performance.